In [95]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import threading
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [13]:
class Croma:
    def __init__(self, search, limit):
        self.search = search
        self.limit = limit
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service = self.s)
        self.driver.get('https://www.croma.com/')
        self.flag = False

        while self.flag != True:
            try:
                #self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
                self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="searchV2"]')))

                self.un.send_keys(self.search)
                self.flag = True
            except (ElementNotInteractableException, TimeoutException) :
                self.driver.refresh()
                time.sleep(3)
                pass
#                 self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
#                 self.un.send_keys(self.search)
#                 self.flag = True
    

    
        time.sleep(2)
        self.un.send_keys(Keys.ENTER)
#         try:
#             self.un = WebDriverWait(self.driver, 12).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
#             #time.sleep(2)
#             self.un.send_keys(search)
#             time.sleep(2)
#             self.un.send_keys(Keys.ENTER)
#         except ElementNotInteractableException:
#             self.driver.refresh()
#             self.un = WebDriverWait(self.driver, 12).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
#             #time.sleep(2)
#             self.un.send_keys(search)
#             time.sleep(2)
#             self.un.send_keys(Keys.ENTER)
            
        
    def link_collection(self):
    
        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        time.sleep(2)
        review_link = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, self.search)))
        #name = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_4rR01T')))


        d = {}
        name = []
        for i in review_link:
            name.append(i.text)

        for i, j in  zip(review_link, name):
            x = self.search.lower()
            if x in j.lower() and '(Renewed)' not in j:
                d[i] = j

        time.sleep(1)
        return d
    
    def scrapings(self, d, limit):
        j = 1
        prod_review= {}
        for i in d:
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            #i.click()
            webdriver.ActionChains(self.driver).move_to_element(i).click(i).perform()
            while len(self.driver.window_handles) != 2:
                time.sleep(3)
                webdriver.ActionChains(self.driver).move_to_element(i).click(i).perform()

            #driver.execute_script("arguments[0].click();", i)
            time.sleep(2)
            #driver.switch_to.window(driver.window_handles[j])
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)
            #time.sleep(2)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(7)
            #try:
            #    all_comments = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'btn-wrap.view-all-review-btn')))
            #all_comments[0].click()
            #   webdriver.ActionChains(driver).move_to_element(all_comments).click(all_comments).perform
            #except TimeoutException:
            #    pass


            #time.sleep(3)

            # scraping review
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')

            h = x.find_all('div', {'class' : 'desc'})
            reviews = set()
            y = d[i]
            for k in h:
                reviews.add(k.text)
                print(k.text)
                if len(reviews) == self.limit:
                    break
            #driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            #curr_height = driver.execute_script('return document.body.scrollHeight;')
            flag = 0
            reviews1 = reviews
            while len(reviews) != self.limit:
                try:
                    #driver.execute_script('window.scrollBy(0, 2000);')
                    #time.sleep(2)
                    flag+=1
                    more = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, 'View All Reviews')))
                    webdriver.ActionChains(self.driver).move_to_element(more).click(more).perform()
                    e = self.driver.page_source
                    x= BeautifulSoup(e, 'html.parser')
                    t = x.find_all('div', {'class' : 'desc'})
                    #print(curr_height)
                    #curr_height += 100
                    for l in t:
                        reviews.add(l.text.strip())
                        if len(reviews) == self.limit:
                            break

                    self.driver.execute_script('window.scrollBy(5350, 100);')
                    time.sleep(3)
                    print(flag)
                    if flag == 11:
                        if len(reviews1) == len(reviews):
                            break

                    #driver.execute_script('window.scrollBy(0, 2000);')
                except TimeoutException:
                    flag+=3
                    if flag >= 11 :
                        break
                    else:
                        pass

            prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])


            print(j)
            j+=1

        return prod_review
        
        
    def run(self):
        main = []
        main_d = {}
        now = time.time()
        for i in range(2):
            d = self.link_collection()
            print(len(d))
            new_d = {}
            for i, j in d.items():
                if i in main_d:
                    continue
                else:
                    new_d[i] = j
            for i, j in d.items():
                main_d[i] = j
            x = self.scrapings(new_d, self.limit)
            main.append(x)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(3)
            next_page =  WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'btn.btn-secondary.btn-viewmore')))
            webdriver.ActionChains(self.driver).move_to_element(next_page).click(next_page).perform()
        return main
#x = Croma('iPhone 14', 10)

    
    

In [45]:
class Flipkart:
    def __init__(self, search, limit):
        self.search = search
        self.limit = limit
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service = self.s)
        self.driver.get('https://www.flipkart.com/')
        
        x_button = WebDriverWait(self.driver, 17).until(EC.presence_of_element_located((By.CLASS_NAME, '_2KpZ6l._2doB4z'))) 
        x_button.click()
        self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')))
        self.un.send_keys(self.search)
        time.sleep(2)
        self.un.send_keys(Keys.ENTER)
        
    
    def link_collection(self):
    
        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        time.sleep(2)
        review = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_1fQZEK')))
        title = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_4rR01T')))

        titles = [] 
        d = {}
        reviews = []

        try:
            for i in review:
                if '(' in i.text:
                    reviews.append(i)
        except StaleElementReferenceException:
            self.driver.refresh()
            time.sleep(2)
            self.link_collection()
        try:
            for i, j in zip(title, reviews):
                #x = search.lower()
                if '(Renewed)' not in i.text:
                    d[j] = i.text
                    titles.append(i.text)
        except StaleElementReferenceException:
            self.driver.refresh()
            time.sleep(2)
            self.link_collection()

        if len(d) == 0:
            self.driver.refresh()
            time.sleep(2)
            self.link_collection()



        time.sleep(1)
        return d
    
    
    def scraping(self, d, limit):
        j = 1
        prod_review= {}
        for i in d:
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            i.click()
            time.sleep(2)
            #driver.switch_to.window(driver.window_handles[j])
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)
            #time.sleep(2)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(7)
            all_comments_link = WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, '_3UAT2v._16PBlm'))) 
            self.driver.execute_script("arguments[0].click();", all_comments_link)

            #time.sleep(3)

            # scraping review
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')

            h = x.find_all('div', {'class' : 't-ZTKy'})
            reviews = set()
            y = d[i]
            for k in h:
                reviews.add(k.text)
                if len(reviews) == limit:
                    break


            if len(reviews) != self.limit or len(reviews) < self.limit:
                    try:
                        while len(reviews) != self.limit:
                            review1 = reviews
                            next_page = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1LKTO3')))
                            if len(next_page) == 1:
                                next_page[0].click()
                            elif len(next_page) == 2:
                                next_page[1].click()
                            time.sleep(4)
                            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                            e = self.driver.page_source
                            x= BeautifulSoup(e, 'html.parser')
                            t = x.find_all('div', {'class' : 't-ZTKy'})
                            for l in t:
                                reviews.add(l.text.strip())
                                if len(reviews) == limit:
                                    break
                            prod_review[y] = reviews

                    except TimeoutException:
                        pass
            else:
                prod_review[y] = reviews

            #prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])


            print(j)
            j+=1

        return prod_review

    
    def run(self):
        main = []
        for i in range(3):
            d = self.link_collection()
            print(len(d))
            x = self.scraping(d, self.limit)
            main.append(x)
            #driver.switch_to.window(driver.window_handles[0])
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            next_page = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1LKTO3')))
            try:
                if len(next_page) == 1:
                    next_page[0].click()
                elif len(next_page) == 2:
                    next_page[1].click()
            except ElementClickInterceptedException:
                self.driver.refresh()
                time.sleep(4)
                self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                
                flag = False
                while flag != True:
                    try:
                        next_page = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1LKTO3')))
                        if len(next_page) == 1:
                            next_page[0].click()
                        elif len(next_page) == 2:
                            next_page[1].click()
                        flag = True
                    except (ElementNotInteractableException, TimeoutException, ElementClickInterceptedException):
                        self.driver.refresh()
                        time.sleep(3)
                        pass
        return main

In [54]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import threading


class Amazon:
    def __init__(self, search, limit, page_visit):
        self.search = search
        self.limit = limit
        self.page_visit = page_visit
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=self.s)
        self.driver.get('https://www.amazon.in/')
        self.un = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="twotabsearchtextbox"]')))
        self.un.send_keys(search)
        time.sleep(2)
        self.un.send_keys(Keys.ENTER)

    def link_collection(self):
        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        first_page_id = self.driver.current_window_handle
        review = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,
                                                                                           'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))  # a-size-base.s-underline-text
        reviews = []
        price = []
        for i in review:
            if '₹' not in i.text:
                reviews.append(i)

        for i in review:
            if '₹' in i.text:
                price.append(i.text)

        try:
            title1 = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-size-base-plus.a-color-base.a-text-normal')))
        except TimeoutException:
            title1 = []

        try:
            title2 = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-size-medium.a-color-base.a-text-normal')))
        except TimeoutException:
            title2 = []

        main_title = title1 if len(title1) > len(title2) else title2

        titles = []
        d = {}
        prices = {}
        for i, j in zip(main_title, reviews):
            x = self.search.lower()
            if ('(Renewed)' not in i.text) or ('WeConnect' not in i.text) or (
                    'Protect+' not in i.text):  # or ('AppleCare' not in i.text)
                d[j] = i.text
                titles.append(i.text)

        for i, j in zip(main_title, price):
            prices[i.text] = j

        time.sleep(1)
        return d, titles, prices

    def scraping(self, d, limit, prices):
        # j = 1
        prod_review = {}
        ratings = {}
        link_of_product = {}
        overall_rating = {}
        for i in d:
            y = d[i]
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            # driver.switch_to.window(first_page_id)
            i.click()
            time.sleep(2)
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)

            link = self.driver.current_url
            link_of_product[y] = link

            #         #current_page_id = driver.current_window_handle

            #         #driver.switch_to.window(current_page_id)
            #         try:
            #             driver.switch_to.window(driver.window_handles[j])
            #         except IndexError:
            #             driver.switch_to.window(driver.window_handles[0])
            #             j+=1
            #             continue

            time.sleep(6)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')

            try:
                all_comments = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'a-link-emphasis.a-text-bold')))
                all_comments.click()
                time.sleep(2)
            except TimeoutException:
                pass

            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            try:
                box = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-dropdown-prompt')))
                for k in box:
                    if k.text == 'All formats':
                        k.click()
            except TimeoutException:
                pass

            try:
                click = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.ID, 'format-type-dropdown_1')))
                click.click()
                time.sleep(3)
            except TimeoutException:
                pass

             # scrapping views
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')

            # scraping rating
            h = x.find_all('span', {'class': 'a-size-medium a-color-base'})

            clean = ' '
            for g in h:
                cleaned = g.text.strip()
                if g.text.strip() != '':
                    clean += cleaned[:3]
            overall_rating[y] = cleaned.strip()

            ## scraping review

            t = x.find_all('span', {'class': 'a-size-base review-text review-text-content'})
            reviews = set()
            for l in t:
                reviews.add(l.text.strip())
                if len(reviews) == self.limit:
                    break

            # star ratings
            h = x.find_all('td', {'class': 'a-text-right a-nowrap'})
            rating = {}
            counter = 5
            for rat in h:
                rating['{} Stars'.format(counter)] = rat.text.strip()
                counter -= 1

            ratings[y] = rating


            if len(reviews) != self.limit or len(reviews) < self.limit:
                try:
                    n = 0
                    while len(reviews) != self.limit:
                        try:
                            last = WebDriverWait(self.driver, 5).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'a-disabled.a-last')))
                            x = last.text
                            if x == 'Next page':
                                break
                        except (TimeoutException, StaleElementReferenceException):
                            review1 = reviews
                            next_ = WebDriverWait(self.driver, 10).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'a-last')))
                            next_.click()
                            time.sleep(4)
                            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                            e = self.driver.page_source
                            x = BeautifulSoup(e, 'html.parser')
                            t = x.find_all('div', {'class': 'a-size-base review-text review-text-content'})
                            for l in t:
                                reviews.add(l.text.strip())
                                if len(reviews) == self.limit:
                                    break
                            prod_review[y] = reviews
                            if len(reviews) == len(review1):
                                break
                except TimeoutException:
                    pass
            else:
                prod_review[y] = reviews

            prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])

        return prod_review, ratings, link_of_product, overall_rating

    def run(self, page_visit):
        review_of_product = []
        ratings_of_product = []
        link_product = []
        price_product = []
        overall_r = []

        for i in range(self.page_visit):
            d, titles, prices = self.link_collection()
            print(titles)
            prod, ratings, links, overall = self.scraping(d, self.limit, prices)

            review_of_product.append(prod)
            ratings_of_product.append(ratings)
            link_product.append(links)
            price_product.append(prices)
            overall_r.append(overall)


            self.driver.switch_to.window(self.driver.window_handles[0])
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(2)
            flag = False
            while flag != True:
                try:
                    next_ = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME,
                                                                                                's-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator')))
                    next_.click()
                    flag = True
                except (TimeoutException, ElementNotInteractableException, ElementClickInterceptedException):
                    self.driver.refresh()
                    time.sleep(3)
                    pass
        return review_of_product, ratings_of_product, link_product, price_product, overall_r



In [55]:
# multithreading
#import threading

x = Croma('iPhone 14', 10)
y = Flipkart('iPhone 14', 10)
z = Amazon('iPhone 14', 10)

thread1 = threading.Thread(target=x.run, name="Thread 1")
thread2 = threading.Thread(target=y.run, name="Thread 2")
thread3 = threading.Thread(target=z.run, name="Thread 3")


# start the threads
thread1.start()
thread2.start()
thread3.start()

# wait for the threads to finish
thread1.join()
thread2.join()
thread3.join()



['Apple iPhone 14 (128 GB) - Blue', 'Apple iPhone 14 Plus (128 GB) - Purple', 'Apple iPhone 14 (128 GB) - Midnight', 'Apple iPhone 14 (128 GB) - Starlight', 'Apple iPhone 14 (128 GB) - Blue', 'Apple iPhone 14 (128 GB) - Yellow', 'Apple iPhone 14 (256 GB) - Purple', 'Apple iPhone 14 (128 GB) - Purple', 'Apple iPhone 14 Plus (128 GB) - Starlight', 'Apple iPhone 14 Plus (128 GB) - Midnight', 'Apple iPhone 14 (256 GB) - Yellow', 'Apple iPhone 14 (256 GB) - (Product) RED', 'Apple iPhone 14 Plus (128 GB) - Purple', 'Apple iPhone 14 (256 GB) - Starlight', 'Apple iPhone 14 (256 GB) - Midnight', 'Apple iPhone 14 (256 GB) - Blue', 'Apple iPhone 14 (512 GB) - Purple', 'Apple iPhone 14 Plus (128 GB) - Blue']20

24
Good one
Its wonderful to have this product with excellent new features
Pathetic Experience
Very nice
Good and helpful
The product is a flagship one without any doubt.
Excellent
It was an amazing experience
Good experience
Screen, Camera ease of use 
1
Good response
Amazing and Nice expe

21
3
22
Very good
Excellent services and product is good and I will invite my friend and family to shop from Croma thank you so much
Nice product
Very comfortable
Excellent.
Good experience
23
24
4
24
1
2
3
4
5
6
7
8
9
5
10
11
12
6
13
14
15
16
17
7
18
19
20
21
8
22
23
24
9
Excellent
Super
Very good
Wonderful phone.
10
Very happy with the product and the offer
Excellent
11
12
13
14
15
16
All good
Good it is
Superb
The Experience was Great.
Products are genuine. Price also affordable and offers attractive Services are very prompt as per customer needs and worth to purchase from Croma. Our Favorite place to shop electronics items.
Excellent product as expected from Apple
Product is excellent.
Love the product
Wonderful experience
Good product in best price and services also good
17
Great product and excellent store.
I loved it
SUPERB
PLEASNT EXPERIENCE. LIKED IT
Nice very nice
Awesome
Awesome experience
Awesome product and professional, experienced sales staff at the store. 
Best display 

In [14]:
x = Croma('iPhone 14', 30)
y = x.run()

20
Got the best deal
Great product
Its great and thanks to the customer care guy Mr Seikh who helped us thoroughly to choose the right product. Thumbs up to team Croma
It was good
Nice price and good service
Good experience
Nice experience
Very nice
Ok good
Nice experience
1
2
3
4
5
6
7
1
Product was amazing
Excellent
Amazing experience
Nice product
It's very bad
Buying experience was good
All good
Good product and very good service by Sagar Agrawal
Love it!
Excellent
1
2
3
4
5
6
7
2
Good response
Amazing and Nice experience.
Awesome product
Fantastic
Amazing
The product is excellent
Product is awesome
Nice performance
Excellent
amazing phone
1
2
3
4
5
6
7
3
Good one
Its wonderful to have this product with excellent new features
Pathetic Experience
Very nice
Good and helpful
The product is a flagship one without any doubt.
Excellent
It was an amazing experience
Good experience
Screen, Camera ease of use 
1
2
3
4
5
6
7
4
Product is very good
Nice good
Very nice
Excellent
Good experience

16
Good one
We are very happy. We like the product very much.
Good product
I am in love
Amazing
Good Product
It was good
It was an amazing experience
Experience of using the product is great excellent also very good service happy.
I have always used Apple product I.e IPhone this is my 3rd iPhone . I have IPad , IPod . The recent iPhone 14 pro max is good the camera is excellent which gives you sharp pictures . The other features like find , switch etc are excellent. The only thing I hated is you have discounted the practice of giving basic asseriores like giving a complete set of charger, ear phone. I would suggest why don't you give the asseriores with built in cost along with the phone .
1
2
17
Very good
Excellent services and product is good and I will invite my friend and family to shop from Croma thank you so much
Nice product
Very comfortable
Excellent.
Good experience
18
Finally after long wait I got my Apple I phone 14 pro max 1TB, gold colour, madly in love with phone performa

In [16]:
y

[{'Apple iPhone 14 (128GB, Blue)': {'Could be better',
   'Excellent',
   'Good',
   'Good 5',
   'Good dealing',
   'Good experience',
   'Good experience to purchase',
   'Good for me',
   'Got the best deal',
   'Great',
   'Great Product',
   'Great experience',
   'Great product',
   'Great product and service.',
   'It was good',
   'It was great.',
   'Its a good product',
   'Its a very good product.',
   'Its great and thanks to the customer care guy Mr Seikh who helped us thoroughly to choose the right product. Thumbs up to team Croma',
   'Nice experience',
   'Nice price and good service',
   'Nice shopping experience. Got the best price.',
   'Ok good',
   'Phone heats up while charging',
   'Satisfied',
   'So far good',
   'Very good',
   'Very good performance',
   'Very nice',
   'Wonderful'},
  'Apple iPhone 14 (128GB, Starlight)': {'All good',
   'Amazing experience',
   'Best service. I am Happy',
   'Buying experience was good',
   'Excellent',
   'Excellent produc

In [25]:
for i in y:
    for j, k in i.items():
        print(j, '--', k)
        print('-------------------------------------')

Apple iPhone 14 (128GB, Blue) -- {'Could be better', 'Good', 'Nice price and good service', 'Good 5', 'Great product and service.', 'So far good', 'Great product', 'Great', 'It was good', 'Its great and thanks to the customer care guy Mr Seikh who helped us thoroughly to choose the right product. Thumbs up to team Croma', 'Nice experience', 'Very nice', 'Very good', 'Phone heats up while charging', 'Great experience', 'Wonderful', 'Its a good product', 'Satisfied', 'Nice shopping experience. Got the best price.', 'It was great.', 'Excellent', 'Good experience', 'Good dealing', 'Good for me', 'Very good performance', 'Good experience to purchase', 'Great Product', 'Ok good', 'Got the best deal', 'Its a very good product.'}
-------------------------------------
Apple iPhone 14 (128GB, Starlight) -- {'Thank you to Mr. Amit for helping and giving a good deal', 'Great', 'All good', 'Very good', 'Not good', 'Good experience great deal', 'good to use', 'Product was amazing', 'Best service. I 

In [42]:
import pandas as pd
import numpy as np
df = pd.DataFrame()


In [43]:
z = 0
df = pd.DataFrame({'Product Name': [], 'Review': []})
for i in y:
    for j, k in i.items():
        for l in k:
            df.loc[z] = [j, l]
            z+=1
df

,Product Name,Review
0,"Apple iPhone 14 (128GB, Blue)",Could be better
1,"Apple iPhone 14 (128GB, Blue)",Good
2,"Apple iPhone 14 (128GB, Blue)",Nice price and good service
3,"Apple iPhone 14 (128GB, Blue)",Good 5
4,"Apple iPhone 14 (128GB, Blue)",Great product and service.
...,...,...
573,"Apple iPhone 14 Pro Max (128GB, Gold)",Excellent services and product is good and I w...
574,"Apple iPhone 14 Pro Max (128GB, Gold)",Excellent.
575,"Apple iPhone 14 Pro Max (128GB, Gold)",Very comfortable
576,"Apple iPhone 14 Pro Max (1TB, Gold)",Finally after long wait I got my Apple I phone...


In [44]:
df

,Product Name,Review
0,"Apple iPhone 14 (128GB, Blue)",Could be better
1,"Apple iPhone 14 (128GB, Blue)",Good
2,"Apple iPhone 14 (128GB, Blue)",Nice price and good service
3,"Apple iPhone 14 (128GB, Blue)",Good 5
4,"Apple iPhone 14 (128GB, Blue)",Great product and service.
...,...,...
573,"Apple iPhone 14 Pro Max (128GB, Gold)",Excellent services and product is good and I w...
574,"Apple iPhone 14 Pro Max (128GB, Gold)",Excellent.
575,"Apple iPhone 14 Pro Max (128GB, Gold)",Very comfortable
576,"Apple iPhone 14 Pro Max (1TB, Gold)",Finally after long wait I got my Apple I phone...


In [53]:
obj = Flipkart('iPhone 14', 30)
z = obj.run()

24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
24
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [56]:
obj1 = Amazon('iPhone 14', 30)
z1 = obj1.run()

['Apple iPhone 14 (128 GB) - Blue', 'Apple iPhone 13 (128GB) - (Product) RED', 'Apple iPhone 14 (128 GB) - Midnight', 'Apple iPhone 14 (128 GB) - Blue', 'Apple iPhone 14 (128 GB) - Starlight', 'Apple iPhone 14 (128 GB) - Yellow', 'Apple iPhone 14 (256 GB) - Yellow', 'Apple iPhone 14 Pro (128 GB) - Silver', 'Apple iPhone 14 (512 GB) - Starlight', 'Apple iPhone 14 (512 GB) - Blue', 'Apple iPhone 14 (512 GB) - (Product) RED', 'Apple iPhone 14 (512 GB) - Purple', 'Apple iPhone 14 Pro (128 GB) - Deep Purple', 'Apple iPhone 14 Plus (256 GB) - Yellow', 'Apple iPhone 14 Pro (256 GB) - Deep Purple', 'Apple iPhone 14 Pro (256 GB) - Space Black', 'Apple iPhone 14 (512 GB) - Midnight', 'Nillkin CamShield Pro Case Compatible with iPhone 14 Pro, [Camera Protection] with Slide Camera Cover, Slim Stylish Protective Case Compatible with iPhone 14 Pro 6.1"-Black']
['Apple iPhone 14 (128 GB) - Yellow', 'Nillkin for iPhone 14 Plus Case with Slide Camera Cover [Lens Plustection] CamShield Plus Anti-Scratch

In [57]:
main_df = df

In [58]:
flipkart_review = z
amazon_review = z1

In [64]:
z1 = 0
df1 = pd.DataFrame({'Product Name': [], 'Review': []})
for i in flipkart_review:
    for j, k in i.items():
        for l in k:
            df1.loc[z1] = [j, l]
            z1+=1
df1

,Product Name,Review
0,"APPLE iPhone 14 (Midnight, 128 GB)",Just awesome!!I switch to iPhone 14 from iPhon...
1,"APPLE iPhone 14 (Midnight, 128 GB)",nice photo but bettery is not good as it show ...
2,"APPLE iPhone 14 (Midnight, 128 GB)",Best smart phone under this price range compar...
3,"APPLE iPhone 14 (Midnight, 128 GB)",Great phone with great capabilities 🔥Excellent...
4,"APPLE iPhone 14 (Midnight, 128 GB)",Reasons to get it over regular 13- 50 percent ...
...,...,...
2155,"APPLE iPhone 12 mini (Black, 128 GB)",My first iphone look like a beautiful value fo...
2156,"APPLE iPhone 12 mini (Black, 128 GB)",As I am iPhone user since 2010 and up till now...
2157,"APPLE iPhone 12 mini (Black, 128 GB)","Very good product thanks to Flipkart, valuable..."
2158,"APPLE iPhone 12 mini (Black, 128 GB)",I switched from an iPhone 6s to a 12 mini and ...


In [65]:
z1 = 0
df2 = pd.DataFrame({'Product Name': [], 'Review': []})
for i in amazon_review:
    for j, k in i.items():
        for l in k:
            df2.loc[z1] = [j, l]
            z1+=1
df2

,Product Name,Review
0,Apple iPhone 14 (128 GB) - Blue,
1,Apple iPhone 14 (128 GB) - Blue,I feel it’s worth investing in iPhone 14 if yo...
2,Apple iPhone 14 (128 GB) - Blue,Not working as I felt. Not finding much differ...
3,Apple iPhone 14 (128 GB) - Blue,I updated the iPhone 14 from iPhone 11.I excha...
4,Apple iPhone 14 (128 GB) - Blue,all good
...,...,...
104,Compatible with iPhone 14 Pro Case 6.1 inch wi...,Très pratique avec la porte coulissante qui pe...
105,Compatible with iPhone 14 Pro Case 6.1 inch wi...,PROTEGE LA CAMARA SUPER BIEN Y ESTA MUY PADRISIMO
106,Compatible with iPhone 14 Pro Case 6.1 inch wi...,Esta resistente y sirve para cuidar que no se ...
107,Compatible with iPhone 14 Pro Case 6.1 inch wi...,Se adapta fácilmente al teléfono y viene con u...


In [71]:
main_df = pd.concat([df, df1, df2], axis = 0)
main_df

,Product Name,Review
0,"Apple iPhone 14 (128GB, Blue)",Could be better
1,"Apple iPhone 14 (128GB, Blue)",Good
2,"Apple iPhone 14 (128GB, Blue)",Nice price and good service
3,"Apple iPhone 14 (128GB, Blue)",Good 5
4,"Apple iPhone 14 (128GB, Blue)",Great product and service.
...,...,...
104,Compatible with iPhone 14 Pro Case 6.1 inch wi...,Très pratique avec la porte coulissante qui pe...
105,Compatible with iPhone 14 Pro Case 6.1 inch wi...,PROTEGE LA CAMARA SUPER BIEN Y ESTA MUY PADRISIMO
106,Compatible with iPhone 14 Pro Case 6.1 inch wi...,Esta resistente y sirve para cuidar que no se ...
107,Compatible with iPhone 14 Pro Case 6.1 inch wi...,Se adapta fácilmente al teléfono y viene con u...


In [72]:
cleaned = main_df.to_csv('product_reviews.csv', index = False, encoding='utf-8') # False: not include index

In [91]:
n = main_df['Review']

In [92]:
import re
cleaned = []
for i in n:
    x = re.sub(r"[^a-zA-Z\s.,!?']", "", i)
    y = re.sub(r'READ MORE','', x)
    #y = re.sub(r'[^\x00-\x7F]+', '', x)
    cleaned.append(y)
    
j = 0
for i in cleaned:
    print(j, i)
    j+=1

0 Could be better
1 Good
2 Nice price and good service
3 Good 
4 Great product and service.
5 So far good
6 Great product
7 Great
8 It was good
9 Its great and thanks to the customer care guy Mr Seikh who helped us thoroughly to choose the right product. Thumbs up to team Croma
10 Nice experience
11 Very nice
12 Very good
13 Phone heats up while charging
14 Great experience
15 Wonderful
16 Its a good product
17 Satisfied
18 Nice shopping experience. Got the best price.
19 It was great.
20 Excellent
21 Good experience
22 Good dealing
23 Good for me
24 Very good performance
25 Good experience to purchase
26 Great Product
27 Ok good
28 Got the best deal
29 Its a very good product.
30 Thank you to Mr. Amit for helping and giving a good deal
31 Great
32 All good
33 Very good
34 Not good
35 Good experience great deal
36 good to use
37 Product was amazing
38 Best service. I am Happy
39 Very good product
40 Excellent
41 Good experience
42 Buying experience was good
43 Amazing experience
44 Exc

1109 Thanks Flipkart service is fine nd it comes in a  days.product is genuine nd I love it
1110 This mine rd IPhone Model iPhone s  Best one when I started android to appleiPhone s  Best Camera Poor Battery Finally iPhone  One of best iPhone i used  fantastic camera  Good battery life. Super fast face lock system. Good charger comparing old iPhones. Happy to having this powerful iPhone.
1111 Review after a week Overall a powerful package for photo, video and gaming. Only thing which could have been better is battery, though it works whole day with low usage , but extensive use it would not go beyond half day.Camera Pictures in full zoom Sound quality Display .Performance 
1112 It's a perfect mobile. Comfortable in hands... Size is good. Looks so classy. For me it's the best
1113 Excellent smart phone, Good battery backup and performance is very good, Good camera quality.
1114 Amazing deviceI switch to iPhone  from iPhone . Great camera and power management full day battery backup. Sou

2259 Perfect phone , Apple after allGo for it eyes closed if in the budget.Camera best quality, crystal clear photos and super stable videos at k  fps,best slow motion in the phone ever.Performance  best then any android even   pro Battery life  last for a day use anyhow.Super smooth software experience.Topnotch security level Very lightweight to hold.
2260 I really m glad that i went for i phone in place of all other android devices like the Samsung S series nd one plus . Also , m amazed by the battery life in this i phone . Single charge and its last for a day easily ..Even the gaming experience was great
2261 Delightful phone, the phone is just a peice of art, sleek, eye catchy, super fast and got everything u need...best one available
2262 Green colour is charming and priceless    No weight this iPhone       Totally mind blowing     Battery backup is not best of best but overall hall day backup normally usage  After update new version battery backup superb hall day easily use it   

In [93]:
n[582]
#cleaned[582]

'Super phone . camera is super😍READ MORE'

In [96]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\HASHIM
[nltk_data]     RAZI\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [99]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\HASHIM
[nltk_data]     RAZI\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [102]:
token = []
stop_words = set(stopwords.words('english'))
for i in cleaned:
    word_tokens = word_tokenize(i)
    filtered_words = [word for word in word_tokens if not word.lower() in stop_words]
    word = ' '.join(filtered_words)
    words = re.sub('[^a-zA-Z0-9 \n]', '', word)
    
    print(words)
    token.append(words)

    

Could better
Good
Nice price good service
Good
Great product service 
far good
Great product
Great
good
great thanks customer care guy Mr Seikh helped us thoroughly choose right product  Thumbs team Croma
Nice experience
nice
good
Phone heats charging
Great experience
Wonderful
good product
Satisfied
Nice shopping experience  Got best price 
great 
Excellent
Good experience
Good dealing
Good
good performance
Good experience purchase
Great Product
Ok good
Got best deal
good product 
Thank Mr Amit helping giving good deal
Great
good
good
good
Good experience great deal
good use
Product amazing
Best service  Happy
good product
Excellent
Good experience
Buying experience good
Amazing experience
Excellent product
Great deal
Nice work
Good product good service Sagar Agrawal
good product Croma nice place purchase electronics supportive staff 
s bad
Superb
Nice
Nice product
Good
Nice product best service
product good
Great
good job
Good
Love 
product excellent
good phone
product good service g

iPhone solid choice looking highquality smartphone experience  camera topnotch  performance smooth  design sleek modern  However  drawbacks consider comparing Android phonesOne biggest cons iPhone lack customization options  Apple strict policy apps features used devices  frustrating users want control pho 
Switched Xiaomi Pro heater means heats lot much cool device also used iPhone little bit better iPhone best device
happy first time phone  experience outstanding phoneBut wanted phone fortunately 
Thanks Flipkart service fine nd comes daysproduct genuine nd love
mine rd IPhone Model iPhone Best one started android appleiPhone Best Camera Poor Battery Finally iPhone One best iPhone used fantastic camera Good battery life  Super fast face lock system  Good charger comparing old iPhones  Happy powerful iPhone 
Review week Overall powerful package photo  video gaming  thing could better battery  though works whole day low usage  extensive use would go beyond half dayCamera Pictures full 

using phone one month  Migrating Android iphone first time  Initially fear battery backup hearing othersI normal user gamerBut  really happy say  best phone awesome battery experience till  charging phone two days  Images crystal clear edit video image using default app  nice product especially iphone plus  Anyone go ahead 
happy use iPhone plus good battery life good camera quality good thank APPLE
Best iPhones till a bigger screen best processor  Refresh could hz disappointing  love start light colour  thought going purple blue attract
go  People comparing pro according s better one screen size much bigger battery life best compared series iphones even pro max ones Yes comes Hz refresh rate wo nt matter regular gamer  Coming camera course getting better quality pro potraits telephoto lens phones give quality sho 
camera expected  requirement good camera smartphone look options better camera phone options available priceIf looking performance  battery  larger screen showoff friends fa

good good  much money go  feel heavy hand  camra  battery performance next level  spend money carefully  thanks Flipkart 
Finally  got iPhone Pro Max GB Deep Purple  Performance  Looks  Feels Quality awesome  Image Video Quality superb without doubt  nt think  go  AWESOME  LOVED 
loved
purchased iphone first time  loved camera quality  would ask apple please provide feature guide  know utilize iphone fully feel like s worth paying lacs rupees 
King  really super se upar flipkart delivery mind blowing  Delivered early  Mine pro max gb Silver colour 
Genuine product
Madly love phone performance blazing fast love pro motion display dynamic island works advertised love feature would like thank Flipkart organising stock delivering phone time delivery date
Satisfied product  
Bought TB pro Max  Op product  Delivered safely effectively  Thank
Awsum Perfect product apple
Amazing
One best Smartphone buy right nowLet s start Pros phoneDynamic Island joyAlwayson screen gorgeousExquisitely premium

happy use iPhone plus good battery life good camera quality good thank APPLE
Best iPhones till a bigger screen best processor  Refresh could hz disappointing  love start light colour  thought going purple blue attract
go  People comparing pro according s better one screen size much bigger battery life best compared series iphones even pro max ones Yes comes Hz refresh rate wo nt matter regular gamer  Coming camera course getting better quality pro potraits telephoto lens phones give quality sho 
camera expected  requirement good camera smartphone look options better camera phone options available priceIf looking performance  battery  larger screen showoff friends family iphone go 
Awesome camera quality
Feels amazing use switching android ios totally new interface processor top notch battery long lasting display amazing sound  Camera phone next level clarity specially videography cinematic mode mind blowing  Really impressed iphone plus
absolute stunner apple  hyped controversial phone

got iphone powerful mobile phone used everSo  go want phone best specification like camera  battery backup  display  bionic chipThat s review phone differ 
Great iPhone snappy experience apple kind  Upgraded iPhone  ProsCamera top class Battery top performed Chipset need say apple kind Security expect apple Display super bright industry leading colour accuracy super responsive Build quality expect apple sturdy premium durable beautiful stylish  Os stable os smartphone industry Cons GDisplay based OLED technology Charger headphones apple stic 
Really satisfied Product received  totally genuine packaging also really good ur planning buy go 
Amazing Powerful Durable GadgetIm happy camera picture quality  Amazing face id unlocked dark room  Strong battery perfect screen size carry easily pocket  third iPhone  shifted android Samsung Note series iPhone strong build quality peace mind next yearsDont think much go suggest go minimum gb variant gb  Ive attached puppy pics fi 
Best budget Iphon

In [106]:
main_df['Cleaned'] = token

In [107]:
main_df

,Product Name,Review,Cleaned
0,"Apple iPhone 14 (128GB, Blue)",Could be better,Could better
1,"Apple iPhone 14 (128GB, Blue)",Good,Good
2,"Apple iPhone 14 (128GB, Blue)",Nice price and good service,Nice price good service
3,"Apple iPhone 14 (128GB, Blue)",Good 5,Good
4,"Apple iPhone 14 (128GB, Blue)",Great product and service.,Great product service
...,...,...,...
104,Compatible with iPhone 14 Pro Case 6.1 inch wi...,Très pratique avec la porte coulissante qui pe...,Trs pratique avec la porte coulissante qui per...
105,Compatible with iPhone 14 Pro Case 6.1 inch wi...,PROTEGE LA CAMARA SUPER BIEN Y ESTA MUY PADRISIMO,PROTEGE LA CAMARA SUPER BIEN ESTA MUY PADRISIMO
106,Compatible with iPhone 14 Pro Case 6.1 inch wi...,Esta resistente y sirve para cuidar que no se ...,Esta resistente sirve para cuidar que se raspe...
107,Compatible with iPhone 14 Pro Case 6.1 inch wi...,Se adapta fácilmente al teléfono y viene con u...,Se adapta fcilmente al telfono viene con un pr...


In [108]:
import pandas as pd
from textblob import TextBlob

def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

main_df['sentiment'] = main_df['Cleaned'].apply(get_sentiment)


,Product Name,Review,Cleaned,sentiment
0,"Apple iPhone 14 (128GB, Blue)",Could be better,Could better,0.500000
1,"Apple iPhone 14 (128GB, Blue)",Good,Good,0.700000
2,"Apple iPhone 14 (128GB, Blue)",Nice price and good service,Nice price good service,0.650000
3,"Apple iPhone 14 (128GB, Blue)",Good 5,Good,0.700000
4,"Apple iPhone 14 (128GB, Blue)",Great product and service.,Great product service,0.800000
...,...,...,...,...
104,Compatible with iPhone 14 Pro Case 6.1 inch wi...,Très pratique avec la porte coulissante qui pe...,Trs pratique avec la porte coulissante qui per...,0.000000
105,Compatible with iPhone 14 Pro Case 6.1 inch wi...,PROTEGE LA CAMARA SUPER BIEN Y ESTA MUY PADRISIMO,PROTEGE LA CAMARA SUPER BIEN ESTA MUY PADRISIMO,0.333333
106,Compatible with iPhone 14 Pro Case 6.1 inch wi...,Esta resistente y sirve para cuidar que no se ...,Esta resistente sirve para cuidar que se raspe...,0.000000
107,Compatible with iPhone 14 Pro Case 6.1 inch wi...,Se adapta fácilmente al teléfono y viene con u...,Se adapta fcilmente al telfono viene con un pr...,0.000000


In [109]:
sentiment = main_df['sentiment']


In [125]:
avg = main_df.groupby('Product Name')['sentiment'].mean().sort_values(ascending = False)[:1]

In [126]:
avg.index

Index(['Apple iPhone 14 Pro Max (128GB, Silver)'], dtype='object', name='Product Name')

In [ ]:
def clean_and_predict(data):
    cleaned = []
    for i in data:
        x = re.sub(r"[^a-zA-Z\s.,!?']", "", i)
        y = re.sub(r'READ MORE','', x)
        #y = re.sub(r'[^\x00-\x7F]+', '', x)
        cleaned.append(y)
        
    token = []
    stop_words = set(stopwords.words('english'))
    for i in cleaned:
        word_tokens = word_tokenize(i)
        filtered_words = [word for word in word_tokens if not word.lower() in stop_words]
        word = ' '.join(filtered_words)
        words = re.sub('[^a-zA-Z0-9 \n]', '', word)

        print(words)
        token.append(words)
    
    main_df['Cleaned'] = token
        
    def get_sentiment(text):
        
        return TextBlob(text).sentiment.polarity

    main_df['sentiment'] = main_df['Cleaned'].apply(get_sentiment)
    sentiment = main_df['sentiment']
    
    avg = main_df.groupby('Product Name')['sentiment'].mean().sort_values(ascending = False)[:1]
    
    best_product = avg.index
    
    return best_product